In [1]:
import requests
import numpy as np


def download_data(url,filename):    #download file as data
    result = requests.get(url)
    result.raise_for_status()
    with open(filename,'wb') as FILE:
        for chunk in result.iter_content(102400):
            FILE.write(chunk)
            
url01 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw4_nnet_train.dat'
url02 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw4_nnet_test.dat'
download_data(url01, 'hw4_nnet_train.txt')
download_data(url02, 'hw4_nnet_test.txt')
    
    
def readout(filename):
    with open(filename,'r') as FILE:
        n, Dx, Dy = 0 ,[], []
        
        for chunk in FILE:
            X= chunk.split()   #split the line into a list of string
            X= [ float(X[j]) for j in range(len(X)) ]  #convert the string to numbers   
#            X.insert(0,1)                #insert a constant as bias or as threshold
            Dx.append(X[0:len(X)-1])     #gather all data ## index len(X)-1 will not be included here
            Dy.append(X[len(X)-1])       #index len(X)-1 only
            n=n+1
        x = np.array(Dx)  #change list X into array x
        y = np.array(Dy)
    print(x,n)            
    return x,y,n        


def eout(w,testx,testy):
    err = []
    yhat = [np.dot(w,testx[i]) for i in range(testn)]

    for a,b in zip(yhat,testy):
        if a*b<=0:
            err.append(1)
        else:
            err.append(0)
    eout = sum(err) /testn
    return eout


trainx, trainy, trainn = readout('hw4_nnet_train.txt')
testx, testy, testn    = readout('hw4_nnet_test.txt') 

[[-0.77947     0.838221  ]
 [ 0.155635    0.895377  ]
 [-0.0599077  -0.71778   ]
 [ 0.207596    0.758933  ]
 [-0.195983   -0.375487  ]
 [ 0.588489   -0.842554  ]
 [ 0.00719859 -0.548316  ]
 [ 0.738839   -0.603394  ]
 [ 0.704648   -0.0204201 ]
 [ 0.969927    0.641371  ]
 [ 0.435431    0.744773  ]
 [-0.844258    0.742354  ]
 [ 0.591425   -0.546021  ]
 [-0.0690931   0.03766   ]
 [-0.951549   -0.733055  ]
 [-0.129881    0.756761  ]
 [-0.495346   -0.566279  ]
 [-0.903994    0.509221  ]
 [ 0.292351    0.16089   ]
 [ 0.647986   -0.779338  ]
 [ 0.375956    0.0782031 ]
 [ 0.24589     0.00451467]
 [-0.457192    0.423905  ]
 [-0.441279    0.705719  ]
 [ 0.507447    0.758726  ]] 25
[[-1.06006e-01 -8.14670e-02]
 [ 1.77930e-01 -3.45951e-01]
 [ 1.02162e-01  7.18258e-01]
 [ 6.94078e-01  6.23397e-01]
 [ 2.35411e-02  7.27432e-01]
 [-3.19728e-01 -8.34114e-01]
 [-1.86744e-01  5.38878e-01]
 [-6.36967e-01  1.52685e-01]
 [-4.74463e-01  8.54344e-01]
 [-3.56277e-02 -2.71588e-01]
 [-1.48603e-01  1.61762e-01]
 [

In [6]:
#11 Back propagation

import random
import numpy as np
import time


def initw(n,m):
#    w = [[random.uniform(-r,r) for i in range(m)] for i in range(n+1)]  #vector W 有M個 --- dim of w1: n+1*m matrix
    w = np.random.rand(n+1, m)
    w = (w-0.5)*2*r
    return np.array(w)

def updatew(w,x,delta):          # Gradient descent: w = w - eta*x(l-1)[i]*delta(l)[j]
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w[i,j] = w[i,j]-eta*x[i]*delta[j]
    return w
    
def err(w1,w2,x,y,n):
    x = np.insert(x,0,np.full(n,1),axis=1)
    x1 = np.tanh(w1.T.dot(x.T))
    x1 = np.insert(x1,0,np.full(n,1),axis=0)
    yhat = np.tanh(w2.T.dot(x1))
    er = y*yhat<0
    eout = er.sum()/n
    

    return eout
  

m = [1,6,11,16,21]
t = 500        #run t experiments
T = 50000    #run T iteratios per experiment
eta = 0.1
r = 0.1
d = len(trainx[0])
N = trainn



for M in m:
    start = time.time()   
    nndim = [d,M,1]   #d-M-1
    Err = 0
    n0 = nndim[0]       #numbers of x0
    n1 = nndim[1]       #numbers of hidden neurons
    n2 = nndim[2]       #numbers of output neuron
 
    for times in range(t):
        w1 = initw(nndim[0],nndim[1])  
        w2 = initw(nndim[1],nndim[2])


        for iteration in range(T):
            randomn = random.randint(0,N-1)
            x0 = np.insert(trainx[randomn],0,1)  #dimension of d+1
            x0 = np.reshape(x0,(-1,1))           #One shape dimension can be -1, inferred from the length of the array and remaining dim
            s1 = w1.T.dot(x0)
            x1 = np.tanh(s1)

            x1 = np.insert(x1,0,1,axis=0)  #dimension of M+1
            s2 = w2.T.dot(x1)
            x2 = np.tanh(s2)

            delta2 = (-8)*(trainy[randomn]-np.tanh(s2))/(np.exp(s2)+np.exp(-s2))
            delta1 = delta2*w2[1:]*4/(np.exp(s1)+np.exp(-s1))

            w2 = updatew(w2,x1,delta2)
            w1 = updatew(w1,x0,delta1)

        Err = Err+err(w1,w2,testx,testy,testn)
#        print(Err/(times+1))
        
    Eout = Err/t
    
    end = time.time()
    print('M =',M,'Eout =',Eout,'time consumed:',end-start)

0.036 
 -30.598546266555786


In [ ]:
a = np.array([[1,2]])
b = np.array([[2,3],[4,5]])

In [ ]:
a.T.dot(a)

In [ ]:
m = [0,1,2]
m[0]

In [ ]:
w = np.array([[1,1]])
w = np.delete(w,0,0)
c = np.append(w,[[0,1],[2,3]],0)